In [5]:
#!pip install twarc
!pip install transformers==3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.6/754.6 kB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.1 requires transformers<4.21,>=4.1, but you have transformers 3.0.0 which is incompatible.


In [6]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from transformers import BertTokenizer, BertModel
import re
from nltk.corpus import stopwords
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'
#from twarc import Twarc
'''data = pd.read_csv('/kaggle/input/the-climate-change-twitter-dataset/The Climate Change Twitter Dataset.csv')
data = data.dropna()
data = data.sample(frac=1)
data = data.iloc[:5000]'''

The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


"data = pd.read_csv('/kaggle/input/the-climate-change-twitter-dataset/The Climate Change Twitter Dataset.csv')\ndata = data.dropna()\ndata = data.sample(frac=1)\ndata = data.iloc[:5000]"

In [7]:
'''consumer_key="mgUcSHDYZ228CbYKyUm9X1Cia"
consumer_secret="WKPkWZSBmb2cwrGYiZZU3dYlXEq93n0hlTyQm8u90HwtRtUZcy"
access_token="1596162900837761025-ibC5Jebjm8bf9hk2W0VeSXpxH6DJu0"
access_token_secret="a0KZOaIcLWDspbOcd167jsxKiTh7E6c7cnbBLW57EPtBc"
content = list()
ids = list()
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

for tweet in t.hydrate(data.id) :
    content.append(tweet['full_text'])
    ids.append(tweet['id'])'''

'consumer_key="mgUcSHDYZ228CbYKyUm9X1Cia"\nconsumer_secret="WKPkWZSBmb2cwrGYiZZU3dYlXEq93n0hlTyQm8u90HwtRtUZcy"\naccess_token="1596162900837761025-ibC5Jebjm8bf9hk2W0VeSXpxH6DJu0"\naccess_token_secret="a0KZOaIcLWDspbOcd167jsxKiTh7E6c7cnbBLW57EPtBc"\ncontent = list()\nids = list()\nt = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)\n\nfor tweet in t.hydrate(data.id) :\n    content.append(tweet[\'full_text\'])\n    ids.append(tweet[\'id\'])'

In [8]:
'''print(len(content))
drop_ids = set(data.id).difference(set(ids))
for i in drop_ids :
    data = data.loc[data['id'] != i]
print(len(data))

data['Content'] = content
data.to_csv('/kaggle/working/tweet-data.csv',index=False)'''

"print(len(content))\ndrop_ids = set(data.id).difference(set(ids))\nfor i in drop_ids :\n    data = data.loc[data['id'] != i]\nprint(len(data))\n\ndata['Content'] = content\ndata.to_csv('/kaggle/working/tweet-data.csv',index=False)"

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
#tweets = pd.read_csv('/kaggle/input/climate-tweets/climate_tweets.csv')
#tweets = tweets.drop(['URL','Likes','Retweets'],axis=1)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [10]:
import inspect
print(inspect.getargspec(model))

ArgSpec(args=['self'], varargs='input', keywords='kwargs', defaults=None)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  


In [11]:
tweets = pd.read_csv('/kaggle/input/tweetdata/tweet-data.csv')
tweets = tweets.drop(['topic','gender','temperature_avg','aggressiveness'],axis=1)
mentions = "^(RT|rt)( @\w*)?[: ]"
hashtags = "#([a-zA-Z0-9_]{1,50})"
punctuations = string.punctuation
stop_words = set(stopwords.words('english'))

# remove @ and # texts
tweets['Content'] = tweets['Content'].apply(lambda x : re.sub(mentions,'',x))
tweets['Content'] = tweets['Content'].apply(lambda x : re.sub(hashtags,'',x))
tweets['Content'] = tweets['Content'].apply(lambda x : x.lower())

#removing digits and punctuations
#tweets['Content'] = tweets['Content'].apply(lambda x : re.sub('[0-9]','',x))        
#tweets['Content'] = tweets['Content'].apply(lambda x : re.sub('[^\w\s]','',x))

#remove stop words
def remove_stopwords(text) :
    cleaned = []
    for word in word_tokenize(text) :
        if word not in stop_words :
            cleaned.append(word)
            
    return ' '.join(cleaned)

#tweets['Content'] = tweets['Content'].apply(remove_stopwords)
print(tweets)

                     created_at                   id         lng        lat  \
0     2017-12-09 18:42:56+00:00   939566047677394944 -123.186770  44.851230   
1     2019-05-18 12:05:32+00:00  1129719667281547265  -71.214540  46.812280   
2     2018-10-08 23:20:52+00:00  1049439513339092992  151.207320 -33.867850   
3     2016-11-01 15:16:57+00:00   793471897639923712 -122.419906  37.779026   
4     2010-10-01 09:59:49+00:00          26065785509    8.801290  48.823980   
...                         ...                  ...         ...        ...   
4361  2019-05-11 03:33:09+00:00  1127054005115342848  -74.006015  40.712728   
4362  2019-05-09 03:56:26+00:00  1126335089338671104  -95.992770  36.153980   
4363  2019-05-14 06:24:02+00:00  1128184174441930752  153.028090 -27.467940   
4364  2016-06-25 07:26:30+00:00   746605472510255104  -85.759410  38.254240   
4365  2018-01-03 15:55:21+00:00   948583570569707521   72.504510  23.014800   

      sentiment    stance                          

In [12]:
#tweets['Content'] = tweets['Content'].apply(lambda x : sent_tokenize(x))


In [13]:

# Create tokenized column
tweets['len'] = tweets['Content'].apply(lambda x : len(x))
tweets = tweets.drop(tweets.index[(tweets['len'] == 0)],axis=0)
tweets['tokenize_ID'] = tweets['Content'].apply(lambda x : tokenizer(x ,padding="max_length", truncation=True))
#tweets['tokens'] = tweets['Content'].apply(lambda x : tokenizer.convert_ids_to_tokens(tokenizer.encode(x)))
tweets

,created_at,id,lng,lat,sentiment,stance,Content,len,tokenize_ID
0,2017-12-09 18:42:56+00:00,939566047677394944,-123.186770,44.851230,0.484304,believer,"@hanamichels like climate change, there’s beli...",79,"[input_ids, token_type_ids, attention_mask]"
1,2019-05-18 12:05:32+00:00,1129719667281547265,-71.214540,46.812280,0.759282,believer,heatwave seems to make manmade climate change...,89,"[input_ids, token_type_ids, attention_mask]"
2,2018-10-08 23:20:52+00:00,1049439513339092992,151.207320,-33.867850,-0.579046,believer,don't blame wildfires on climate change – it's...,107,"[input_ids, token_type_ids, attention_mask]"
3,2016-11-01 15:16:57+00:00,793471897639923712,-122.419906,37.779026,-0.413409,neutral,ojo! in 1912 scientists warned of us coal and...,114,"[input_ids, token_type_ids, attention_mask]"
4,2010-10-01 09:59:49+00:00,26065785509,8.801290,48.823980,-0.066502,believer,influencers! have you registered for @climate...,203,"[input_ids, token_type_ids, attention_mask]"
...,...,...,...,...,...,...,...,...,...
4361,2019-05-11 03:33:09+00:00,1127054005115342848,-74.006015,40.712728,0.428099,believer,does anyone think global warming is a good th...,112,"[input_ids, token_type_ids, attention_mask]"
4362,2019-05-09 03:56:26+00:00,1126335089338671104,-95.992770,36.153980,-0.467969,neutral,yea your nudes fire but what are u doing to s...,76,"[input_ids, token_type_ids, attention_mask]"
4363,2019-05-14 06:24:02+00:00,1128184174441930752,153.028090,-27.467940,-0.646242,believer,"impakterdotcom ""rt sierraclub: ""pre-disaster m...",182,"[input_ids, token_type_ids, attention_mask]"
4364,2016-06-25 07:26:30+00:00,746605472510255104,-85.759410,38.254240,0.048020,believer,"“the important question for australia, then, i...",301,"[input_ids, token_type_ids, attention_mask]"


In [17]:
for i,stance in enumerate(tweets.stance) :
    if stance == 'neutral':
        tweets['stance'].iloc[i] = [0,1,0]
    elif stance == 'believer' :
        tweets['stance'].iloc[i] = [0,0,1]
    elif stance == 'denier' :
        tweets['stance'].iloc[i] = [1,0,0]

train, test, evaluate = np.split(tweets.sample(frac=1), [3999,4182])
class BertDataset(Dataset) :
    def __init__(self,data) :
        self.data = data
        self.tokenized = self.data.tokenize_ID
        self.label = self.data.stance
    
    def __len__(self) :
        return len(self.tokenized)
    
    def __getitem__(self,idx) :
        if torch.is_tensor(idx) :
            idx = idx.to_list()
            
        item = self.tokenized.iloc[idx]
        label = self.label.iloc[idx]
        sample = {
                  "input_ids" : torch.tensor(item['input_ids'],dtype=torch.long),
                  "token_type_ids" : torch.tensor(item['token_type_ids'],dtype=torch.long),
                  "attention_mask" : torch.tensor(item['attention_mask'],dtype=torch.long),
                  "label" : torch.tensor(label,dtype=torch.long)
                }

        return sample

training_data = BertDataset(train)
testing_data = BertDataset(test)

train_dataloader = DataLoader(training_data, batch_size=4, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=4, shuffle=True)

In [11]:
for param in model.parameters():
    param.requires_grad = False
    
class BERT(torch.nn.Module):
    def __init__(self,model):
        super(BERT, self).__init__()
        self.l1 = model
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 3)
    
    def forward(self, ids, mask):
        _, output_1= self.l1(input_ids=ids, attention_mask=mask)
        output_1 = torch.tensor(output_1)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output
    

model = BERT(model=model)
model.to(device)

BERT(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
         

In [21]:
def loss_fn(outputs, targets):
    return torch.nn.CrossEntropyLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-5)

def train(epoch):
    model.train()
    for _,data in enumerate(train_dataloader, 0):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['label'].to(device, dtype = torch.float)

        outputs = model(ids, mask)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [22]:
for epoch in range(4):
    train(epoch)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


Epoch: 0, Loss:  0.21144327521324158
Epoch: 1, Loss:  1.4781180620193481
Epoch: 2, Loss:  0.6440751552581787
Epoch: 3, Loss:  0.6523970365524292


In [3]:
from datetime import datetime
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(test_dataloader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['label'].to(device, dtype = torch.float)
            outputs = model(ids, mask)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [46]:
from sklearn import metrics

outputs, targets = validation(epoch)
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
recall = metrics.recall_score(targets, outputs, average='micro')
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"Recall Score = {recall}")
print(f"F1 Score = {f1_score_micro}")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


Accuracy Score = 0.7704918032786885
Recall Score = 0.7704918032786885
F1 Score = 0.7704918032786885


In [35]:
for i,stance in enumerate(test.stance) :
    if stance == [0,1,0]:
        test['stance'].iloc[i] = 0
    elif stance == [0,0,1] :
        test['stance'].iloc[i] = 1
    elif stance == [1,0,0] :
        test['stance'].iloc[i] = -1

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [36]:
from datetime import datetime
test['created_at'] = test['created_at'].astype('datetime64[ns]')
test.sort_values(by='created_at', inplace = True)

In [38]:
import plotly.graph_objects as go

fig = go.Figure([go.Scatter(x=test['created_at'], y=test['stance'])])
fig.show()